In [ ]:
import torch
from torchvision import transforms
from PIL import Image
from TransMIL import * 
from TransMIL.models import TransMIL

# Assume you have a function to load the model and it's been trained or a pre-trained model is loaded
model = TransMIL()
model.load_state_dict(torch.load('trans_mil_model.pth'))
model.eval()

# Image preprocessing
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize if necessary
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

def classify_patch(image_path):
    image = Image.open(image_path).convert('RGB')
    image = transform(image).unsqueeze(0)  # Add batch dimension
    with torch.no_grad():
        outputs = model(image)
        _, predicted = torch.max(outputs, 1)
    return predicted.item()


# create file directory structure and train-test split (one time run when the file structure is not created)
main_dir = "/media/ist/Drive2/MANSOOR/Neuroimaging-Project/Breast_Cancer_Classification_Project"

tiles_dir =  f"{main_dir}/test_tiles" 
save_model_path = f"{main_dir}/WSI_Breast_Cancer_Classification/Model_Weights"

# Example usage
image_path = f'{tiles_dir}/n_14_test/SUB_n_14_tile_(6_38).png'
prediction = classify_patch(image_path)
print("Predicted class:", "Benign" if prediction == 0 else "Malignant")
